<br>

# Introdução

Atualiza o valor da UFESP


In [ ]:
import sys
import pandas as pd
from pathlib import Path
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


In [ ]:
# Paths
project_path = Path().absolute().parent
print(project_path)
sys.path.append(str(project_path))


In [ ]:
from ufesp.paths import data_path


In [ ]:
URL = 'https://legislacao.fazenda.sp.gov.br/Paginas/ValoresDaUFESP.aspx'
dfs = pd.read_html(URL)
df = dfs[2]
df.head()


In [ ]:
df['VALOR EM R$'] = df['VALOR EM R$'].replace(',', '', regex=True)
df['VALOR EM R$'] = df['VALOR EM R$'].replace('\u200b', '', regex=True)
df['VALOR EM R$'] = df['VALOR EM R$'].astype(float)
df['VALOR EM R$'] = df['VALOR EM R$'] / 100
df.head()


In [ ]:
df['PERÍODO'] = df['PERÍODO'].replace('de ', '', regex=True)
df['PERÍODO'] = df['PERÍODO'].replace(' a ', ' A ', regex=True)

df['PERÍODO'] = df['PERÍODO'].replace('\u200b', '', regex=True)
df['PERÍODO'] = df['PERÍODO'].replace('\xa0', ' ', regex=True)
df[['data_inicio', 'data_fim']] = df['PERÍODO'].str.split('A', 1, expand=True)

df['data_inicio'] = df.loc[:, 'data_inicio'].str.strip()
df['data_inicio'] = pd.to_datetime(df['data_inicio'], format='%d/%m/%Y')

df['data_fim'] = df.loc[:, 'data_fim'].str.strip()
df['data_fim'] = pd.to_datetime(df['data_fim'], format='%d/%m/%Y')

df.loc[:, 'ano_mes'] = pd.to_datetime(df['data_inicio']).dt.to_period('M')

df.rename(
    {'VALOR EM R$': 'valor', 'BASE LEGAL': 'base_legal'}, axis=1, inplace=True
)

df['base_legal'] = df['base_legal'].replace('\u200b', '', regex=True)
df['base_legal'] = df.loc[:, 'base_legal'].str.strip()


In [ ]:
# Results
df = df[[
    'data_inicio',
    'data_fim',
    'ano_mes',
    'valor',
    'base_legal'
]].copy()

# Save
df.to_csv(
    data_path / 'ufesp.csv',
    index=False,
    decimal=',',
)

df
